In [486]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
import datetime
from dateutil.parser import parse
from datetime import datetime
from tqdm import tqdm

# 1. Data Preparation

## 1.1 Global Innovation Index (GII)

### 1.1.1 Load GII data

In [487]:
# Load GII data
GII_data = pd.read_csv('data/GII_dataset.csv')

In [488]:
GII_data.head()

,country,indicator,measure,Unit,Date,Value
0,Argentina,Global Innovation Index,Score,"Score, 100=Max strength 0=Weakest",2013,37.66
1,Argentina,Global Innovation Index,Score,"Score, 100=Max strength 0=Weakest",2014,35.13
2,Argentina,Global Innovation Index,Score,"Score, 100=Max strength 0=Weakest",2015,34.30
3,Argentina,Global Innovation Index,Score,"Score, 100=Max strength 0=Weakest",2016,30.24
4,Argentina,Global Innovation Index,Score,"Score, 100=Max strength 0=Weakest",2017,32.00


### 1.1.2 Data Cleaning

In [489]:
# Make a copy of the data
GII_data_ = GII_data.copy()

In [490]:
# Drop columns that are not needed
GII_data_.drop(['measure', 'Unit'], axis=1, inplace=True)

# Change columns types
GII_data_['Date'] = pd.to_datetime(GII_data_['Date'], format="%Y")
GII_data_['Value'] = pd.to_numeric(GII_data_['Value'])

In [491]:
# pivot the table to reshape it
GII_df_pivot = GII_data_.pivot_table(index=['country', 'Date'], columns='indicator', values='Value')
GII_df_pivot.reset_index(inplace=True)
GII_df_pivot.columns.name = None
GII_df_pivot.head(20)

,country,Date,Business sophistication,Creative outputs,Global Innovation Index,Human capital and research,Infrastructure,Institutions,Knowledge and technology outputs,Market sophistication
0,Argentina,2013-01-01,34.20,47.46,37.66,36.66,35.02,50.68,25.65,37.32
1,Argentina,2014-01-01,32.88,36.92,35.13,38.25,37.98,49.06,25.22,37.73
2,Argentina,2015-01-01,36.26,36.52,34.30,37.67,38.20,48.03,22.24,35.94
3,Argentina,2016-01-01,30.84,25.27,30.24,37.29,43.29,47.21,17.97,35.68
4,Argentina,2017-01-01,33.63,27.63,32.00,42.58,46.60,46.37,17.61,37.74
5,Argentina,2018-01-01,31.38,23.63,30.70,35.53,43.37,54.70,17.88,37.79
6,Argentina,2019-01-01,32.60,24.00,31.90,38.70,45.80,56.70,19.20,37.90
7,Argentina,2020-01-01,26.90,19.60,28.30,35.90,39.50,54.30,17.20,34.60
8,Argentina,2021-01-01,26.70,21.90,29.80,37.00,42.50,52.80,18.70,37.50
9,Argentina,2022-01-01,31.20,24.20,28.60,30.50,44.00,47.60,19.00,24.90


In [492]:
# Rename columns to be more descriptive and easier to work with.
rows = []
for _, row in tqdm(GII_df_pivot.iterrows(), total=GII_df_pivot.shape[0]):
    row_data = dict(
        country=row['country'],
        year=row['Date'],
        gii_index=row['Global Innovation Index'],
        gii_institutions=row['Institutions'],
        gii_human_capital_research=row['Human capital and research'],
        gii_infrastructure=row['Infrastructure'],
        gii_market_sophistication=row['Market sophistication'],
        gii_business_sophistication=row['Business sophistication'],
        gii_knowledge_technology_outputs=row['Knowledge and technology outputs'],
        gii_creative_outputs=row['Creative outputs'],
    )
    rows.append(row_data)

# Create a new dataframe with the new columns
GII_copy_df = pd.DataFrame(rows)

100%|██████████| 190/190 [00:00<00:00, 37555.03it/s]


In [493]:
GII_copy_df.set_index(['country', 'year'], inplace=True)

In [494]:
GII_copy_df.head()

gii_index  gii_institutions  gii_human_capital_research  \
country   year                                                                  
Argentina 2013-01-01      37.66             50.68                       36.66   
          2014-01-01      35.13             49.06                       38.25   
          2015-01-01      34.30             48.03                       37.67   
          2016-01-01      30.24             47.21                       37.29   
          2017-01-01      32.00             46.37                       42.58   

                      gii_infrastructure  gii_market_sophistication  \
country   year                                                        
Argentina 2013-01-01               35.02                      37.32   
          2014-01-01               37.98                      37.73   
          2015-01-01               38.20                      35.94   
          2016-01-01               43.29                      35.68   
          2017-01-01               46.60                      37.74   

                      gii_business_sophistication  \
country   year                                      
Argentina 2013-01-01                        34.20   
          2014-01-01                        32.88   
          2015-01-01                        36.26   
          2016-01-01                        30.84   
          2017-01-01                        33.63   

                      gii_knowledge_technology_outputs  gii_creative_outputs  
country   year                                                                
Argentina 2013-01-01                             25.65                 47.46  
          2014-01-01                             25.22                 36.92  
          2015-01-01                             22.24                 36.52  
          2016-01-01                             17.97                 25.27  
          2017-01-01                             17.61                 27.63

In [495]:
# Add missing rows for Turkey for 2022 (GII data is not available for 2022)
GII_copy_df.loc[('Turkey', pd.to_datetime('2022-01-01'))] = [38.1, 46.1, 38.9, 49.2, 41.6, 32.5, 27.4, 41.5]

In [496]:
GII_copy_df.isnull().sum()

gii_index                           0
gii_institutions                    0
gii_human_capital_research          0
gii_infrastructure                  0
gii_market_sophistication           0
gii_business_sophistication         0
gii_knowledge_technology_outputs    0
gii_creative_outputs                0
dtype: int64

In [497]:
# Save the data to a csv file for later use
GII_copy_df.to_csv('data/GII_dataset_v2.csv', index=True)

In [498]:
GII_df = pd.read_csv('data/GII_dataset_v2.csv', index_col=['country', 'year'], parse_dates=['year'])

In [499]:
GII_df.index.names

FrozenList(['country', 'year'])

In [500]:
GII_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 190 entries, ('Argentina', Timestamp('2013-01-01 00:00:00')) to ('United States of America', Timestamp('2022-01-01 00:00:00'))
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   gii_index                         190 non-null    float64
 1   gii_institutions                  190 non-null    float64
 2   gii_human_capital_research        190 non-null    float64
 3   gii_infrastructure                190 non-null    float64
 4   gii_market_sophistication         190 non-null    float64
 5   gii_business_sophistication       190 non-null    float64
 6   gii_knowledge_technology_outputs  190 non-null    float64
 7   gii_creative_outputs              190 non-null    float64
dtypes: float64(8)
memory usage: 13.4+ KB


In [501]:
GII_df.describe()

,gii_index,gii_institutions,gii_human_capital_research,gii_infrastructure,gii_market_sophistication,gii_business_sophistication,gii_knowledge_technology_outputs,gii_creative_outputs
count,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000
mean,45.166895,68.912684,45.837263,50.754579,56.867263,41.269000,36.916842,38.289684
std,10.823114,15.306705,12.599887,9.887084,12.663938,10.970911,13.253066,11.257063
min,26.500000,37.240000,19.980000,27.490000,24.900000,17.500000,14.600000,17.500000
25%,35.342500,55.347500,35.550000,42.500000,48.000000,31.400000,24.875000,28.005000
50%,46.200000,66.240000,47.150000,52.150000,55.100000,40.170000,34.600000,39.055000
75%,54.950000,84.132500,56.960000,59.637500,64.847500,50.475000,49.060000,47.640000
max,62.400000,93.260000,67.400000,67.140000,87.090000,64.500000,60.800000,62.530000


In [502]:
from linearmodels.panel import PanelOLS

# Set the model formula
formula = 'gii_index ~ 1 + gii_institutions + gii_human_capital_research + gii_infrastructure + gii_market_sophistication + gii_business_sophistication + gii_knowledge_technology_outputs + gii_creative_outputs + EntityEffects'

# Create the model
model = PanelOLS.from_formula(formula=formula, data=GII_df)

# Fit the model
results = model.fit(cov_type='clustered', cluster_entity=True)

results

Dep. Variable:,gii_index,R-squared:,0.9999
Estimator:,PanelOLS,R-squared (Between):,1.0000
No. Observations:,190,R-squared (Within):,0.9999
Date:,"Mon, May 01 2023",R-squared (Overall):,1.0000
Time:,01:02:49,Log-likelihood,438.71
Cov. Estimator:,Clustered,,
,,F-statistic:,1.583e+05
Entities:,19,P-value,0.0000
Avg Obs:,10.0000,Distribution:,"F(7,164)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,2.208e+05
